<a href="https://colab.research.google.com/github/ParkSongJi/BPE-WPT/blob/main/%EA%B3%BC%EC%A0%9C_WPT.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#  "WordPiece Tokenizer"

* wordpiece tokenizer는 위에 설명한 subword tokenizer의 종류 중 하나이다. subword tokenizer에서 대표적으로 사용되는 방법으로 BPE(Byte Pair Encoding) 방법이 있다.

* 일반적으로 많이 사용하는 Sentencepiece의 경우 빈도수를 기반으로 BPE를 수행하며, Wordpiece의 경우 likelihood를 기반으로 BPE를 수행한 알고리즘이다.

* BERT의 경우 Wordpiece를 이용한 tokenizer를 사용하였고, sentencepiece를 사용한 모델 또한 많다. 선택에 따라 필요한 tokenizer를 활용할 수 있다.

In [ ]:
corpus = [
    "This is the Hugging Face course.",
    "This chapter is about tokenization.",
    "This section shows several tokenizer algorithms.",
    "Hopefully, you will be able to understand how they are trained and generate tokens.",
]

In [ ]:
# 사전 토큰화에 사용할 토크나이저로 bert-base-cased 를 사용
from transformers import AutoTokenizer

tokenizer = AutoTokenizer.from_pretrained("bert-base-cased")

# 입력받은 corpus 를 바탕으로 단어의 구성을 확인하는 과정

In [ ]:
# 분석하고자 하는 문장의 빈도수 확인
# defaultdict는 디폴트 값을 가지는 딕셔너리를 생성함
from collections import defaultdict

# 임의의 딕셔너리를 int 형태로 기본값 0으로 하여 만듦
word_freqs = defaultdict(int)

# 기존의 데이터를 전부 소문자로 바꾸는 작업 // 특수문자 없애는 작업
corpus = list(map(str.lower, corpus))
corpus = [text.replace(',', '').replace('.', '') for text in corpus]

for text in corpus:
    # words_with_offsets 변수 안에 반복문으로 들어온 문장들의 단어들에게 인덱스를 부여
    # [('This', (0, 4)), ('is', (5, 7)), ('the', (8, 11)), ('Hugging', (12, 19)), ('Face', (20, 24)), ('course', (25, 31)), ('.', (31, 32))]
    words_with_offsets = tokenizer.backend_tokenizer.pre_tokenizer.pre_tokenize_str(text)

    # 토큰화 한 변수를 가지고 와서 각 단어만 저장
    new_words = [word for word, offset in words_with_offsets]

    # 이후 사전에 만들어 놓은 word_freqs 변수에 저장
    for word in new_words:
        word_freqs[word] += 1
word_freqs

defaultdict(int,
            {'this': 3,
             'is': 2,
             'the': 1,
             'hugging': 1,
             'face': 1,
             'course': 1,
             'chapter': 1,
             'about': 1,
             'tokenization': 1,
             'section': 1,
             'shows': 1,
             'several': 1,
             'tokenizer': 1,
             'algorithms': 1,
             'hopefully': 1,
             'you': 1,
             'will': 1,
             'be': 1,
             'able': 1,
             'to': 1,
             'understand': 1,
             'how': 1,
             'they': 1,
             'are': 1,
             'trained': 1,
             'and': 1,
             'generate': 1,
             'tokens': 1})

# 입력받은 corpus 를 바탕으로 vocaburlary 를 만드는 과정

In [ ]:
alphabet = []

# word_freqs 변수 안에 있는 단어들의 첫 글자를 추출하여 해당글자가 alphabet 안에 포함되어 있지 않다면 추가
for word in word_freqs.keys():
    if word[0] not in alphabet:
        alphabet.append(word[0])

    # 각 알파벳의 첫 글자를 제외한 나머지 글자들을 추출하여 ## 을 붙이고 여기서 alphabet 안에 포함되어 있지 않다면 추가
    for letter in word[1:]:
        if f"##{letter}" not in alphabet:
            alphabet.append(f"##{letter}")

alphabet.sort()
alphabet

['##a',
 '##b',
 '##c',
 '##d',
 '##e',
 '##f',
 '##g',
 '##h',
 '##i',
 '##k',
 '##l',
 '##m',
 '##n',
 '##o',
 '##p',
 '##r',
 '##s',
 '##t',
 '##u',
 '##v',
 '##w',
 '##y',
 '##z',
 'a',
 'b',
 'c',
 'f',
 'g',
 'h',
 'i',
 's',
 't',
 'u',
 'w',
 'y']

In [ ]:
# 단어가 존재하지 않거나 오류가 발생했을시 사용할 수 있는 특수 조건 삽입
vocab = ["[PAD]", "[UNK]", "[CLS]", "[SEP]", "[MASK]"] + alphabet.copy()

In [ ]:
# 위에서 만들어 놓은 단어 별 빈도수의 디셔너리 word_freqs 를 가지고 와서 첫 번째 알파벳를 제외하고 나머지 알파벳들에게 ## 을 붙임
splits = {
    word: [c if i == 0 else f"##{c}" for i, c in enumerate(word)]
    for word in word_freqs.keys()
}
print(splits)

{'this': ['t', '##h', '##i', '##s'], 'is': ['i', '##s'], 'the': ['t', '##h', '##e'], 'hugging': ['h', '##u', '##g', '##g', '##i', '##n', '##g'], 'face': ['f', '##a', '##c', '##e'], 'course': ['c', '##o', '##u', '##r', '##s', '##e'], 'chapter': ['c', '##h', '##a', '##p', '##t', '##e', '##r'], 'about': ['a', '##b', '##o', '##u', '##t'], 'tokenization': ['t', '##o', '##k', '##e', '##n', '##i', '##z', '##a', '##t', '##i', '##o', '##n'], 'section': ['s', '##e', '##c', '##t', '##i', '##o', '##n'], 'shows': ['s', '##h', '##o', '##w', '##s'], 'several': ['s', '##e', '##v', '##e', '##r', '##a', '##l'], 'tokenizer': ['t', '##o', '##k', '##e', '##n', '##i', '##z', '##e', '##r'], 'algorithms': ['a', '##l', '##g', '##o', '##r', '##i', '##t', '##h', '##m', '##s'], 'hopefully': ['h', '##o', '##p', '##e', '##f', '##u', '##l', '##l', '##y'], 'you': ['y', '##o', '##u'], 'will': ['w', '##i', '##l', '##l'], 'be': ['b', '##e'], 'able': ['a', '##b', '##l', '##e'], 'to': ['t', '##o'], 'understand': ['u', '##

In [ ]:
# 빈도의 점수를 계산하는 함수
def compute_pair_scores(x):
    # 임의의 딕셔너리 2개를 만들어 놓음
    letter_freqs = defaultdict(int) # 빈도수
    pair_freqs = defaultdict(int)   # 문자 쌍

    # 딕셔너리 안에있는 단어와 그 빈도수를 가지고 옴
    for word, freq in word_freqs.items():

        # 함수에 입력된 딕셔너리(x)에 해당 단어에 대한 분리 결과를 가져옴
        split = x[word]

        # 만약 단어의 길이가 1일 경우에는 단어의 빈도수를 만들어 놓은 딕셔너리에 추가
        if len(split) == 1:
            letter_freqs[split[0]] += freq
            continue

        # 단어의 길이 -1 의 병합결과가 나오는것을 고려
        for i in range(len(split) - 1):

            # 나오는 순서대로 병합과정을 pair 라는 변수에 넣음
            pair = (split[i], split[i + 1])

            # 위와 같이 알파벳별 빈도수를 만들어 놓은 딕셔너리에 추가
            letter_freqs[split[i]] += freq

            # 위에서 생성한 pair 변수를 만들어 놓은 딕셔너리에 추가
            pair_freqs[pair] += freq
        # 마지막 알파벳의 빈도수도 더해줌, 마지막 알파벳은 합칠게 없어서 빈도수 추가가
        letter_freqs[split[-1]] += freq

    # 이후 생성된 빈도수, 문자 쌍 딕셔너리를 바탕으로 병합별 빈도 비중 계산
    scores = {
        pair: freq / (letter_freqs[pair[0]] * letter_freqs[pair[1]])
        for pair, freq in pair_freqs.items()
    }
    return scores

In [ ]:
# 이후 만들어진 함수가 잘 작동하는지 앞에 위치한 병합 5개만 뽑음
# 함수에 단어별로 분석된 딕셔너리 split을 넣고 임의의 변수에 넣어줌
pair_scores = compute_pair_scores(splits)

# 스코어 값이 나온 변수를 인덱스 번호와 함께 반복문을 돌림
# 이때 생성된 pair_scores 딕셔너리는 병합된 조합이 들어있는 형태
for i, key in enumerate(pair_scores.keys()):
    print(f"{key}: {pair_scores[key]}")
    if i >= 5:
        break

('t', '##h'): 0.0625
('##h', '##i'): 0.03409090909090909
('##i', '##s'): 0.02727272727272727
('i', '##s'): 0.1
('##h', '##e'): 0.011904761904761904
('h', '##u'): 0.06666666666666667


In [ ]:
# 이제 위에서 생성한 병합들의 스코어 중 가장 값이 큰 병합을 찾음
# 가장 큰 스코어를 가진 병합을 넣을 변수와 스코어 값을 넣을 변수를 생성
best_pair = ""
max_score = None

# 이어있는 스코어 변수 안에 모든 병합을 순차적으로 삽입, 넘어서면 교체하는 방식
for pair, score in pair_scores.items():
    if max_score is None or max_score < score:
        best_pair = pair
        max_score = score

print(best_pair, max_score)

('a', '##b') 0.2


In [ ]:
# 가장 큰 값을 가진 병합인 ab를 추가
vocab.append("ab")

In [ ]:
# 이제 추가한 병합이 가지는 알파벳 부분을 합치고 업데이트하는 과정
def merge_pair(a, b, x):
    # 단어별 빈도수를 가지고 있는 딕셔너리인 word_ferqs 의 단어들을 하나씩 딕셔너리(x)에 해당 단어에 대한 분리 결과를 가져옴
    for word in word_freqs:
        split = x[word]
        # print(split) # ['t', '##h', '##i', '##s']
        # 해당 단어의 길이가 1인경우 아무 작업없이 넘어감
        if len(split) == 1:
            continue
        # 해당 단어가 여러 부분 단어로 분리된 경우, split 리스트를 순회하면서 a와 b가 인접한 경우를 찾음
        i = 0
        while i < len(split) - 1:
            # 만약 인접할경우 뒤에 나오는 알파벳의 ## 을 제거하고 병합해줌
            if split[i] == a and split[i + 1] == b:
                merge = a + b[2:] if b.startswith("##") else a + b
                # 이후 기존의 split 에서 합쳐진 부분을 마지한 업데이트 split을 생성
                split = split[:i] + [merge] + split[i + 2 :]
            else:
                i += 1
        # 완성된 결과를 최신화
        x[word] = split
    return x

In [ ]:
# merge 후 업데이트 된 부분 확인하는 작업
splits = merge_pair("a", "##b", splits)
splits["about"]

['ab', '##o', '##u', '##t']

In [ ]:
# 목표로하는 vocab 크기를 설정 후 채워질때까지 반복
vocab_size = 70
while len(vocab) < vocab_size:
    # 점수를 측정하는 함수에 splits를 넣고 반복
    scores = compute_pair_scores(splits)
    best_pair, max_score = "", None
    for pair, score in scores.items():
        if max_score is None or max_score < score:
            best_pair = pair
            max_score = score
    # 이후 점수가 가장 큰 병합을 splits에 업데이트
    splits = merge_pair(*best_pair, splits)

    # 가장 큰 병햡을 vocab에 채우는 과정
    new_token = (
        # best_pair[0] : 병합할 앞의 알파벳, best_pair[1][2:] : 병합할 뒤의 알파벳(##삭제)
        best_pair[0] + best_pair[1][2:]
        if best_pair[1].startswith("##")
        else best_pair[0] + best_pair[1]
    )
    # 나온 토큰(병합 구조)를 단어장에 추가
    vocab.append(new_token)

In [ ]:
print(vocab)

['[PAD]', '[UNK]', '[CLS]', '[SEP]', '[MASK]', '##a', '##b', '##c', '##d', '##e', '##f', '##g', '##h', '##i', '##k', '##l', '##m', '##n', '##o', '##p', '##r', '##s', '##t', '##u', '##v', '##w', '##y', '##z', 'a', 'b', 'c', 'f', 'g', 'h', 'i', 's', 't', 'u', 'w', 'y', 'ab', '##fu', 'fa', 'fac', '##ct', '##ful', '##full', '##fully', '##hm', '##thm', 'is', '##thms', '##pt', '##apt', '##hapt', 'chapt', '##za', '##zat', 'abl', '##izat', '##izati', '##cti', '##iz', '##ithms', 'wi', 'wil', 'will', 'al', '##al', 'alg']


In [ ]:
def encode_word(x):
    # 인코딩된 부분 단어를 저장할 리스트 초기화
    tokens = []
    # word의 길이가 0보다 큰 동안 반복
    while len(x) > 0:
        # i를 현재 word의 길이로 초기화
        i = len(x)
        # i가 0보다 크고, word의 처음부터 길이 i까지의 부분 단어가 단어장에 없을 때 반복
        while i > 0 and x[:i] not in vocab:
            # i를 감소시킴
            i -= 1
        # 만약 i가 0이면, 단어장에 없는 부분 단어라는 의미
        if i == 0:
            # "[UNK]" (unknown token)을 반환하고 함수 종료
            return ["[UNK]"]
        # 단어장에 있는 부분 단어를 tokens 리스트에 추가
        tokens.append(x[:i])
        # 인코딩한 부분 단어를 제외한 나머지 부분을 word로 설정
        x = x[i:]
        # word의 길이가 0보다 크면
        if len(x) > 0:
            # 나머지 부분 단어에 "##"를 추가하여 다음 인코딩 준비
            x = f"##{x}"
    # 모든 부분 단어를 인코딩한 tokens 리스트 반환
    return tokens

# 완성된 과정 확인
* 위에서 만든 함수와 과정들이 제대로 작동하는지 확인해 보겠습니다

In [ ]:
print(encode_word("Hugging"))
print(encode_word("hugging"))

['[UNK]']
['h', '##u', '##g', '##g', '##i', '##n', '##g']


In [ ]:
def tokenize(text):
    # 입력받은 문장을 리스트로 변환하는 과정
    pre_tokenize_result = tokenizer._tokenizer.pre_tokenizer.pre_tokenize_str(text)
    pre_tokenized_text = [word for word, offset in pre_tokenize_result]

    # pre_tokenized_text에 대해 encode_word 함수를 사용하여 각 단어를 부분 단어로 인코딩
    encoded_words = [encode_word(word) for word in pre_tokenized_text]
    return sum(encoded_words, [])

In [ ]:
tokenize("This is the Hugging Face course!")

['[UNK]',
 'is',
 't',
 '##h',
 '##e',
 '[UNK]',
 '[UNK]',
 'c',
 '##o',
 '##u',
 '##r',
 '##s',
 '##e',
 '[UNK]']